
**Group 7: Parthiv Borgohain, Anthony Moreno, Jason Nania, Nicole Pham-Nguyen, Kyle Tobia (8:30-10:30AM Cohort)**




In [1]:
import numpy as np
import pandas as pd
import requests

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#from google.colab import drive
import nltk 
nltk.download('punkt')
from nltk import word_tokenize
import pandas as pd
import csv
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt
import numpy as np
import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.manifold import MDS
from matplotlib import pyplot as plt
import sklearn.datasets as dt
from nltk.corpus import stopwords
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer
import seaborn as sns         
from sklearn.metrics.pairwise import manhattan_distances, euclidean_distances
from matplotlib.offsetbox import OffsetImage, AnnotationBbox


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [2]:
# Use this code if using Colab to run Selenium

# Make sure to go to Runtime -> Change runtime and set GPU as hardware accelerator

# !kill -9 -1 # Use this line to delete this VM and start a new one. 
# The above line deletes all files and folders from the current VM and allocates a new one.

#Selenium is an open-source tool that automates web browsers.
!pip install selenium
!apt-get -q update   #Used to handle installation and removal of softwares and libraries
!apt install -yq chromium-chromedriver #ChromeDriver is a separate executable that Selenium WebDriver uses to control Chrome.
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
#WebDriver is a browser automation framework that works with open source APIs. 
#The framework operates by accepting commands, sending those commands to a browser, and interacting with applications.
chrome_options = webdriver.ChromeOptions()
#headless means running chrome with chrome.exe
chrome_options.add_argument('--headless')

chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',options=chrome_options)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 7.7 MB/s 
     |████████████████████████████████| 140 kB 43.8 MB/s 
     |████████████████████████████████| 384 kB 40.6 MB/s 
     |████████████████████████████████| 58 kB 5.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:14 

In [3]:
test = webdriver.Chrome('chromedriver',options=chrome_options)
test.get('https://www.beeradvocate.com/beer/top-rated/')

**Part A** (as discussed in class, we are fetching about 2.2k reviews of Top 250 brands - other reviews do not have any text)

In [4]:
import re
elems = test.find_elements("xpath","//a[@href]")
list_of_urls=[]
url_list=[]
for elem in elems:
    if re.search("https://www.beeradvocate.com/beer/profile/[0-9]+/[0-9]+",elem.get_attribute("href")):
      url_list.append(elem.get_attribute("href"))

url_list = list(set(url_list)) #removing duplicate URLs
url_list = sorted(url_list)#sorting list - so that it does not change with runtime

In [5]:
from lxml import html
import requests

list_of_urls=[]

for url in url_list:
  if url != 'https://www.beeradvocate.com/beer/profile/313/1545/':
    list_of_urls.append(url)


ratings=[]
comments=[]
brand_raw=[]
brands=[]
for url in list_of_urls:
# To add brand into for loop. Add it to the beginning of each rating or comment. Then can split first word out in the dataframe later.

  page = requests.get(url)

  tree = html.fromstring(page.content)  
    
  # Get element using XPath
  rating=(tree.xpath('//*[@id="rating_fullview_content_2"]/span[1]/b[1]/text()'))
  comment=(tree.xpath('//*[@id="rating_fullview_content_2"]/div/text()')) # SEPARATING COMMENTS ON PARAGRAPH BREAKS
  ratings.append(rating)
  comments.append(comment)

cmt=[]
for i in comments:
  for j in i:
    cmt.append(j)

rat=[]
for i in ratings:
  for j in i:
    rat.append(j)

c_df = pd.DataFrame(cmt)
c_df
r_df = pd.DataFrame(ratings)



#initializing an empty dataframe for storing comments data
#reviews = pd.DataFrame(columns = ['brand','rating','comment']) 
#reviews


In [6]:
def check(X):
  if X[0]=='\n':
    return 1
  else:
    return 0

c_df['flag']=c_df[0].map(check)

#combining paragraph breaks into one review comment
comments=[]
i=0
while(i<(len(c_df))):
  if c_df.iloc[i,1]==0:
    lst=[]
    lst.append(c_df.iloc[i,0])
    for j in range(i+1,len(c_df)):
      if c_df.iloc[j,1]==1:
        lst.append(c_df.iloc[j,0])
        if j==len(c_df)-1:
          i=j
          break
      else:
        i=j-1
        break
    i=i+1
    comments.append(lst)

for sentence in comments:
  if len(sentence)>1:
    for x in sentence:
      " ".join(x)

for sentence in comments:
    sentence=str(sentence)


new_comments=[]
for item in comments:
  new_comments.append(str(item).lstrip("'['").rstrip("']'"))



In [7]:
tr=[]
sr=[]
brands=[]
for link in list_of_urls:
  page = requests.get(link)
  tree = html.fromstring(page.content)
  
  TR=[]
  SR=[]
  length=0
  total_rating = tree.xpath('//*[@id="rating_fullview_content_2"]/span[2]/text()')
  TR.append(total_rating)
  for i in TR:
    for j in i:
      tr.append(j)
      length=length+1

  sub_ratings = tree.xpath('//*[@id="rating_fullview_content_2"]/span[5]/text()')
  SR.append(sub_ratings)
  for i in SR:
    for j in i:
      sr.append(j)  

  brand_raw = tree.xpath('//*[@id="XenForo"]/head/title/text()')
  separator = '|'
  brand_raw = ''.join(brand_raw)
  brand = brand_raw.split(separator, 1)[0]
  for i in range(0,length):
    brands.append(brand)

In [8]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'
   
# Create the pandas DataFrame
df = pd.DataFrame(new_comments, columns=['Review'])
df['Brand'] = brands
df['Total Rating'] = tr
df['Look Rating'] = sr
df[['Look Rating','Smell Rating','Taste Rating','Feel Rating','Overall Rating']] = df['Look Rating'].str.split('|',n=-1,expand=True)

def secondpart(Y):
  B=Y.split(':')
  return float(B[1])

df['Look Rating']=df['Look Rating'].map(secondpart)
df['Smell Rating']=df['Smell Rating'].map(secondpart)
df['Taste Rating']=df['Taste Rating'].map(secondpart)
df['Feel Rating']=df['Feel Rating'].map(secondpart)
df['Overall Rating']=df['Overall Rating'].map(secondpart)

def convert_to_float(X):
  return float(X)

print(color.BOLD + "Scraped Reviews:" + color.END)
df['Total Rating'] = df['Total Rating'].map(convert_to_float) #converting rating to float number
df

Scraped Reviews:


,Review,Brand,Total Rating,Look Rating,Smell Rating,Taste Rating,Feel Rating,Overall Rating
0,Smokey,Kaggen! Stormaktsporter,4.50,4.50,4.50,4.50,4.50,4.50
1,"Had low expectations. But, my gawd! This beer ...",Kaggen! Stormaktsporter,4.94,5.00,4.75,5.00,5.00,5.00
2,"Excellent Porter. One of the best. Roasty, com...",Kaggen! Stormaktsporter,4.25,4.25,4.25,4.25,4.25,4.25
3,Pours jet black with a small tan head. Intense...,Kaggen! Stormaktsporter,4.63,4.50,4.50,4.75,4.75,4.50
4,"Bottle, RBESG 2007 in Glasgow, Grand Tasting. ...",Kaggen! Stormaktsporter,4.44,4.75,4.50,4.50,4.25,4.25
...,...,...,...,...,...,...,...,...
2221,"""In this modern era of IPAs; this doesn't hold...",Pliny The Elder,4.00,4.00,4.00,4.00,4.00,4.00
2222,"Russian River Pliny the Elder Imperial IPA, 8%...",Pliny The Elder,4.60,4.50,4.50,4.75,4.00,4.75
2223,Took nearly 20 years to finally get ahold of a...,Pliny The Elder,4.30,4.25,4.25,4.25,4.25,4.50
2224,Poured from a bottle (bought in early Feb 2022...,Pliny The Elder,4.68,4.25,5.00,4.50,4.75,4.75


In [30]:
output1=df.rename({'Brand':'product_name','Review':'product_review','Total Rating':'user_rating'})
output1[['Brand','Review','Total Rating']].to_csv('Scraping Output.csv')

**Part B. Word Frequency Analysis**

In [9]:
# PART B
bdf = df
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk

# lower case for all the reviews
def lowerize(x):
  list1 = []
  for i in x:
    list1.append(str.lower(i))
  return list1
bdf['Review'] = bdf['Review'].str.lower()

# Creating a column for reviews with the stopwords removed
nltk.download('stopwords')
stop = stopwords.words('english')
bdf['no_stopwords'] = bdf['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Removing punctuation from no_stopwords
bdf['no_stopwords'] = bdf['no_stopwords'].str.replace(r'[^\w\s]+', '')

# Tokenizing the original Review column and the the no_stopwords column
bdf['tokenized'] = bdf['Review'].apply(word_tokenize)
bdf['tokenized_review'] = bdf['Review'].apply(word_tokenize)
bdf['tokenized_nostop'] = bdf['no_stopwords'].apply(word_tokenize)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: The default value of regex will change from True to False in a future version.


In [10]:
#find the frequency of each non-stopword
unsorted_frequency = bdf.tokenized_nostop.str.get_dummies(',').sum()
sorted_frequency = unsorted_frequency.sort_values(ascending = False)

In [11]:
# data frame of attributes
columns = ['frequency']
attributes_df = pd.DataFrame(sorted_frequency[:50], columns = columns)

# cleaning up the index formatting
attributes_df.index = attributes_df.index.str.replace("'",'')
attributes_df.index = attributes_df.index.str.replace(" ",'')
attributes_df

,frequency
head,1092
n,877
beer,767
taste,605
carbonation,526
lacing,495
chocolate,492
good,490
nose,480
dark,472


**Part C. Cosine Similarity**

In [12]:
# PART C
# list of attributes
attributes = attributes_df.index.values.tolist() 

att_list_df = pd.read_csv('attributes.csv')

# attributes compiled
att_list = att_list_df['attributes'].values[0] + " " + att_list_df['attributes'].values[1] + " " + att_list_df['attributes'].values[2]

# adding the attributes to the dataframe
bdf['User_Attribute_Input'] = [att_list for i in bdf.index]

In [13]:
import warnings
warnings.filterwarnings('ignore') #suppressing warnings in output

# Calculating BOW cosine similarity scores based on user input attributes
bdf['BOW Similarity Score'] = 0
for i in range(len(bdf)):
  X = bdf['no_stopwords'].loc[i]
  Y = bdf['User_Attribute_Input'].loc[0]
  documents = [X,Y]
  count_vectorizer = CountVectorizer(stop_words = 'english')
  sparse_matrix = count_vectorizer.fit_transform(documents)
  doct_term_matrix = sparse_matrix.todense()
  m2_df = pd.DataFrame(doct_term_matrix, columns = count_vectorizer.get_feature_names(), index = ['x','y'])
  bdf['BOW Similarity Score'].loc[i] = cosine_similarity(m2_df,m2_df)[0,1]

In [14]:
# Output for part C
output_df = bdf[['Brand','Review','BOW Similarity Score']]
output_df.to_csv('Part C Output.csv')

In [15]:
!pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 7.7 MB/s 
     |████████████████████████████████| 127 kB 59.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.12
    Uninstalling urllib3-1.26.12:
      Successfully uninstalled urllib3-1.26.12
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.5.0 requires urllib3[socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.


**Part D. Sentiment Score through VADER**

In [16]:
#Part D
# import SentimentIntensityAnalyzer class
# from vaderSentiment.vaderSentiment module.
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# function to print sentiments
# of the sentence.
def sentiment_scores(sentence):
	sid_obj = SentimentIntensityAnalyzer()
	sentiment_score = sid_obj.polarity_scores(sentence)['compound']
	return sentiment_score

# function calling
bdf['Sentiment Score'] = bdf['Review'].map(sentiment_scores)
print(color.BOLD + "Sentiment Scores are stored in the column Sentiment Score of the below table:" + color.END)
bdf

Sentiment Scores are stored in the column Sentiment Score of the below table:


,Review,Brand,Total Rating,Look Rating,Smell Rating,Taste Rating,Feel Rating,Overall Rating,no_stopwords,tokenized,tokenized_review,tokenized_nostop,User_Attribute_Input,BOW Similarity Score,Sentiment Score
0,smokey,Kaggen! Stormaktsporter,4.50,4.50,4.50,4.50,4.50,4.50,smokey,[smokey],[smokey],[smokey],crispy foam gross,0.0,0.0000
1,"had low expectations. but, my gawd! this beer ...",Kaggen! Stormaktsporter,4.94,5.00,4.75,5.00,5.00,5.00,low expectations but gawd beer blew away easil...,"[had, low, expectations, ., but, ,, my, gawd, ...","[had, low, expectations, ., but, ,, my, gawd, ...","[low, expectations, but, gawd, beer, blew, awa...",crispy foam gross,0.0,0.8639
2,"excellent porter. one of the best. roasty, com...",Kaggen! Stormaktsporter,4.25,4.25,4.25,4.25,4.25,4.25,excellent porter one best roasty complex bold ...,"[excellent, porter, ., one, of, the, best, ., ...","[excellent, porter, ., one, of, the, best, ., ...","[excellent, porter, one, best, roasty, complex...",crispy foam gross,0.0,0.9660
3,pours jet black with a small tan head. intense...,Kaggen! Stormaktsporter,4.63,4.50,4.50,4.75,4.75,4.50,pours jet black small tan head intense roast b...,"[pours, jet, black, with, a, small, tan, head,...","[pours, jet, black, with, a, small, tan, head,...","[pours, jet, black, small, tan, head, intense,...",crispy foam gross,0.0,0.1779
4,"bottle, rbesg 2007 in glasgow, grand tasting. ...",Kaggen! Stormaktsporter,4.44,4.75,4.50,4.50,4.25,4.25,bottle rbesg 2007 glasgow grand tasting pitch ...,"[bottle, ,, rbesg, 2007, in, glasgow, ,, grand...","[bottle, ,, rbesg, 2007, in, glasgow, ,, grand...","[bottle, rbesg, 2007, glasgow, grand, tasting,...",crispy foam gross,0.0,0.7810
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2221,"""in this modern era of ipas; this doesn't hold...",Pliny The Elder,4.00,4.00,4.00,4.00,4.00,4.00,in modern era ipas hold hype get wrong great i...,"[``, in, this, modern, era, of, ipas, ;, this,...","[``, in, this, modern, era, of, ipas, ;, this,...","[in, modern, era, ipas, hold, hype, get, wrong...",crispy foam gross,0.0,0.8463
2222,"russian river pliny the elder imperial ipa, 8%...",Pliny The Elder,4.60,4.50,4.50,4.75,4.00,4.75,russian river pliny elder imperial ipa 8 abv p...,"[russian, river, pliny, the, elder, imperial, ...","[russian, river, pliny, the, elder, imperial, ...","[russian, river, pliny, elder, imperial, ipa, ...",crispy foam gross,0.0,0.8593
2223,took nearly 20 years to finally get ahold of a...,Pliny The Elder,4.30,4.25,4.25,4.25,4.25,4.50,took nearly 20 years finally get ahold bottle ...,"[took, nearly, 20, years, to, finally, get, ah...","[took, nearly, 20, years, to, finally, get, ah...","[took, nearly, 20, years, finally, get, ahold,...",crispy foam gross,0.0,0.9888
2224,poured from a bottle (bought in early feb 2022...,Pliny The Elder,4.68,4.25,5.00,4.50,4.75,4.75,poured bottle bought early feb 2022 pint glass...,"[poured, from, a, bottle, (, bought, in, early...","[poured, from, a, bottle, (, bought, in, early...","[poured, bottle, bought, early, feb, 2022, pin...",crispy foam gross,0.0,0.9929


**Part E. Total Score**

In [17]:
#Part E
#Defining Total Score as the average of Similarity and Sentiment Scores
total_score = (bdf['BOW Similarity Score'] + bdf['Sentiment Score'])/2
bdf['Total Score (BOW Model)'] = total_score
grouped_by_bow=bdf.groupby('Brand')[['Total Rating','BOW Similarity Score','Sentiment Score','Total Score (BOW Model)']].mean() #aggregating similarity and sentiment scores at brand level
print(color.BOLD + "Top 3 Recommendations based on Bag of Words Model (Total Score is computed by taking the average of BOW Similarity Score and Sentiment Score):" + color.END)
grouped_by_bow.sort_values(by = 'Total Score (BOW Model)',ascending=False)[:3] #Finding Top 3 Brands based on Total Score (average of Similarity and Sentiment Scores)

Top 3 Recommendations based on Bag of Words Model (Total Score is computed by taking the average of BOW Similarity Score and Sentiment Score):


,Total Rating,BOW Similarity Score,Sentiment Score,Total Score (BOW Model)
Brand,,,,
Aurelian Lure,4.3900,0.062622,0.93540,0.499011
Triple Shot,4.4525,0.024754,0.95490,0.489827
Flora Plum,4.6940,0.005276,0.93772,0.471498


In [18]:
#Installing SpaCy
!conda install -c conda-forge spacy
!python -m spacy download en_core_web_md

/bin/bash: conda: command not found
2022-10-01 23:48:50.219574: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42.8 MB 1.2 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


**Part F. SpaCY**

In [19]:
#Part F

# New dataframe for part F (creating a druplicate dataframe)
fdf = bdf

#Importing SpaCy
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_md')

# user attribute inputs
#user_input1 = input("Name one attribute: ")
#user_input2 = input("Name a second attribute: ")
#user_input3 = input("Name a third attribute: ")

# attributes compiled
#att_list = user_input1 + " " + user_input2 + " " + user_input3

# adding the attributes to the dataframe
#fdf['User_Attribute_Input'] = [att_list for i in fdf.index]

fdf['token_F'] = df['Review'].apply(lambda x: nlp(x))
fdf['tokenized_input_F'] = df['User_Attribute_Input'].apply(lambda x: nlp(x))
fdf[:5]

,Review,Brand,Total Rating,Look Rating,Smell Rating,Taste Rating,Feel Rating,Overall Rating,no_stopwords,tokenized,tokenized_review,tokenized_nostop,User_Attribute_Input,BOW Similarity Score,Sentiment Score,Total Score (BOW Model),token_F,tokenized_input_F
0,smokey,Kaggen! Stormaktsporter,4.50,4.50,4.50,4.50,4.50,4.50,smokey,[smokey],[smokey],[smokey],crispy foam gross,0.0,0.0000,0.00000,(smokey),"(crispy, foam, gross)"
1,"had low expectations. but, my gawd! this beer ...",Kaggen! Stormaktsporter,4.94,5.00,4.75,5.00,5.00,5.00,low expectations but gawd beer blew away easil...,"[had, low, expectations, ., but, ,, my, gawd, ...","[had, low, expectations, ., but, ,, my, gawd, ...","[low, expectations, but, gawd, beer, blew, awa...",crispy foam gross,0.0,0.8639,0.43195,"(had, low, expectations, ., but, ,, my, gawd, ...","(crispy, foam, gross)"
2,"excellent porter. one of the best. roasty, com...",Kaggen! Stormaktsporter,4.25,4.25,4.25,4.25,4.25,4.25,excellent porter one best roasty complex bold ...,"[excellent, porter, ., one, of, the, best, ., ...","[excellent, porter, ., one, of, the, best, ., ...","[excellent, porter, one, best, roasty, complex...",crispy foam gross,0.0,0.9660,0.48300,"(excellent, porter, ., one, of, the, best, ., ...","(crispy, foam, gross)"
3,pours jet black with a small tan head. intense...,Kaggen! Stormaktsporter,4.63,4.50,4.50,4.75,4.75,4.50,pours jet black small tan head intense roast b...,"[pours, jet, black, with, a, small, tan, head,...","[pours, jet, black, with, a, small, tan, head,...","[pours, jet, black, small, tan, head, intense,...",crispy foam gross,0.0,0.1779,0.08895,"(pours, jet, black, with, a, small, tan, head,...","(crispy, foam, gross)"
4,"bottle, rbesg 2007 in glasgow, grand tasting. ...",Kaggen! Stormaktsporter,4.44,4.75,4.50,4.50,4.25,4.25,bottle rbesg 2007 glasgow grand tasting pitch ...,"[bottle, ,, rbesg, 2007, in, glasgow, ,, grand...","[bottle, ,, rbesg, 2007, in, glasgow, ,, grand...","[bottle, rbesg, 2007, glasgow, grand, tasting,...",crispy foam gross,0.0,0.7810,0.39050,"(bottle, ,, rbesg, 2007, in, glasgow, ,, grand...","(crispy, foam, gross)"


In [20]:
def get_vec_sim(doc1, doc2):    
        return doc1.similarity(doc2)

fdf['Word2vec Similarity'] = fdf.apply(lambda x: get_vec_sim(x['token_F'],x['tokenized_input_F']),axis=1)

total_score = (fdf['Word2vec Similarity'] + bdf['Sentiment Score'])/2
fdf['Total Score (W2V Model)'] = total_score
grouped_by_w2v=fdf.groupby('Brand')[['Word2vec Similarity','Sentiment Score','Total Score (W2V Model)']].mean() #aggregating similarity and sentiment scores at brand level
print(color.BOLD + "Top 3 Recommendations based on Word2Vec Model (Total Score is computed by taking the average of W2V Similarity Score and Sentiment Score):" + color.END)
grouped_by_w2v.sort_values(by = 'Total Score (W2V Model)',ascending=False)[:3] #Finding Top 3 Brands based on Total Score (average of Word2vec Similarity and Sentiment Scores)

Top 3 Recommendations based on Word2Vec Model (Total Score is computed by taking the average of W2V Similarity Score and Sentiment Score):


,Word2vec Similarity,Sentiment Score,Total Score (W2V Model)
Brand,,,
Triple Shot,0.379559,0.954900,0.667230
Double Dry Hopped Double Mosaic Daydream,0.362389,0.930325,0.646357
Aurelian Lure,0.345921,0.935400,0.640660


In [21]:
#Merging both tables so that Scores from BOW Model and W2V Models can be compared
df_merged=grouped_by_bow.merge(grouped_by_w2v, left_on = 'Brand',right_on = 'Brand')
df_merged=df_merged.rename({'Sentiment Score_x':'Sentiment Score'},axis='columns')
df_merged=df_merged[['BOW Similarity Score','Sentiment Score','Total Score (BOW Model)','Word2vec Similarity','Total Score (W2V Model)','Total Rating']]

In [22]:
#Top 3 brands by BOW Model are -
X1=df_merged.sort_values(by='Total Score (BOW Model)',ascending=False)[:3]
print(color.BOLD + "Top 3 Recommendations based on BOW Model:" + color.END)
print(X1[['Total Score (BOW Model)','Total Score (W2V Model)']])

#Top 3 brands by W2V Model are -
X2=(df_merged.sort_values(by='Total Score (W2V Model)',ascending=False))[:3]
print(color.BOLD + "\n\nTop 3 Recommendations based on W2VModel:" + color.END)
print(X2[['Total Score (BOW Model)','Total Score (W2V Model)']])

Top 3 Recommendations based on BOW Model:
                Total Score (BOW Model)  Total Score (W2V Model)
Brand                                                           
Aurelian Lure                  0.499011                  0.64066
Triple Shot                    0.489827                  0.66723
Flora Plum                     0.471498                  0.64045


Top 3 Recommendations based on W2VModel:
                                           Total Score (BOW Model)  \
Brand                                                                
Triple Shot                                               0.489827   
Double Dry Hopped Double Mosaic Daydream                  0.465162   
Aurelian Lure                                             0.499011   

                                           Total Score (W2V Model)  
Brand                                                               
Triple Shot                                               0.667230  
Double Dry Hopped Double Mosaic 

After making our recommendations using both Bag of Words (BoW) and Word to Vector (W2V) models, we can see that typically the same brands receive higher overall scores in the W2V model. This can be attributed to the similarity score being higher in W2V because the same sentiment score was used for each total score. This is most likely due to the fact that the BoW model only references the words in the corpus to calculate similarity, whereas W2V uses pre-trained word vectors with access to much more words to compute its similarity score. We can also see that two of the top 3 recommendations are the same for each, but the difference in score is closer in the BoW model.

**Part G**

In [23]:
# Part G
gdf = df_merged

print(color.BOLD + "Recommendations based on Rating (Table 1):" + color.END)
display(gdf.sort_values(by = 'Total Rating', ascending = False)[:3])

print(color.BOLD + "\n\nRecommendations based on BOW Model (Table 2):" + color.END)
display(grouped_by_bow.sort_values(by = 'Total Score (BOW Model)',ascending=False)[:3])

print(color.BOLD + "\n\nRecommendations based on W2V Model (Table 3):" + color.END)
display(grouped_by_w2v.sort_values(by = 'Total Score (W2V Model)',ascending=False)[:3])

Recommendations based on Rating (Table 1):


,BOW Similarity Score,Sentiment Score,Total Score (BOW Model),Word2vec Similarity,Total Score (W2V Model),Total Rating
Brand,,,,,,
The Adjunct Trail - Bourbon Barrel-Aged,0.00000,0.81260,0.406300,0.361317,0.586958,4.930000
Twice the Daily Serving: Raspberry,0.00616,0.75505,0.380605,0.330141,0.542596,4.833333
Art,0.00000,0.32525,0.162625,0.321496,0.323373,4.810000




Recommendations based on BOW Model (Table 2):


,Total Rating,BOW Similarity Score,Sentiment Score,Total Score (BOW Model)
Brand,,,,
Aurelian Lure,4.3900,0.062622,0.93540,0.499011
Triple Shot,4.4525,0.024754,0.95490,0.489827
Flora Plum,4.6940,0.005276,0.93772,0.471498




Recommendations based on W2V Model (Table 3):


,Word2vec Similarity,Sentiment Score,Total Score (W2V Model)
Brand,,,
Triple Shot,0.379559,0.954900,0.667230
Double Dry Hopped Double Mosaic Daydream,0.362389,0.930325,0.646357
Aurelian Lure,0.345921,0.935400,0.640660


The recommendations are completely different, and the sentiment scores are all lower for the highest rated beers, when comparing them to the 3 beers recommended using user input attributes, which were all greater than .9, whereas the top 3 highest beers had sentiment scores lower than .81. Additionally, the third highest beer (Art) has a much lower sentiment score (.325) than the other two (.812, .755). Although the 3 top rated beers had similar similarity scores (W2V) to the ones in Table 3, we can see that the scores are cumulatively higher for the ones that took into account a user's preferences. Comparing these recommendations to part D, the similarity scores of these 3 beers were all lower than .1, whereas in the initial recommendation, the similarity scores were all greater than .3. Lastly, but most importantly, while some of the similarity scores are relatively comparable, what differentiates the beers that take into account one's preferences is the total scores that we calculated. Once we output the results from the BOW and W2V models we can see that 0.406 is not higher than any of the combined sentiment and similarity scores in the recommendations made by the BOW or W2V models. This indicates that simply selecting the 3 most highly rated beers does not mean you are going to provide a specific user with the beer that fits their tastes best. This highlights the importance of taking customer preferences into considerations when making recommendations. Customers may like the top rated recommendations, but they will not be as satisfied as they would have been if they were recommended a beer that took into account what their unique preferences were.

**Part H. Lift Matrix**

In [24]:
# Part H
from math import sqrt
hdf=bdf
print(attributes_df[:10])
print(color.BOLD + "\n\nSo, taste, carbonation, chocolate, dark are the top 4 attributes" + color.END)
#taste, carbonation, chocolate, dark (top 4 attributes)

             frequency
head              1092
n                  877
beer               767
taste              605
carbonation        526
lacing             495
chocolate          492
good               490
nose               480
dark               472


So, taste, carbonation, chocolate, dark are the top 4 attributes


In [25]:
def words(X): #function to be used to return word length to map function
    return len(X)
hdf['word_count']=hdf['tokenized'].map(words)
total_sentences = len(hdf)
total_sentences

2226

In [26]:
list_of_attributes=['taste', 'carbonation', 'chocolate', 'lacing']
tenBeers = gdf.groupby('Brand').sum('Total Rating').sort_values('Total Rating', ascending = False)[:10]

#lift matrix for attributes and beer
lift_matrix_attributes=pd.DataFrame(columns=list_of_attributes)


lift_matrix_attributes['Index']=tenBeers.index.values
lift_matrix_attributes=lift_matrix_attributes.set_index('Index')
#created a square matrix to store brand-and-feature list values

In [27]:
from math import sqrt
list_of_brands=list(tenBeers.index.values) #storing the top brand names in a list
size_lift_matrix_features_rows=int(len(list_of_brands)) #storing no. of rows of lift matrix (for iteration purpose)
size_lift_matrix_features_columns=int(len(list_of_attributes)) #storing no. of rows of lift matrix (for iteration purpose)

def tolistfunc(X):
  A=[]
  A.append(X)
  return A


hdf['list_brand'] = hdf['Brand'].map(tolistfunc)
hdf['tokenized_brand'] = hdf['tokenized'] + hdf['list_brand']

In [28]:
N=total_sentences

i=0
j=0
for i in range(0,size_lift_matrix_features_rows):
    for j in range(0,size_lift_matrix_features_columns):
        Na=0
        Nb=0
        Nab=0
        for k in range(0,len(hdf)):
            if list_of_brands[i] in hdf['tokenized_brand'][k]:
                  Na=Na+1
            if list_of_attributes[j] in hdf['tokenized_brand'][k]:
                  Nb=Nb+1
            if (list_of_brands[i] in hdf['tokenized_brand'][k]) and (list_of_attributes[j] in hdf['tokenized_brand'][k]):
                  Nab=Nab+1       
        lift_matrix_attributes.iat[i,j]=(N*Nab)/(Na*Nb) 

In [29]:
print(color.BOLD + "The Required Lift Matrix is given below:\n\n" + color.END)
lift_matrix_attributes

The Required Lift Matrix is given below:




,taste,carbonation,chocolate,lacing
Index,,,,
The Adjunct Trail - Bourbon Barrel-Aged,0.0,0.0,0.0,0.0
Twice the Daily Serving: Raspberry,1.208469,0.0,0.0,0.0
Art,0.906352,0.0,0.0,0.0
Barrel Aged Imperial German Chocolate Cupcake Stout,0.517915,0.0,2.569697,0.639839
Truth - Vanilla Bean,0.659165,0.383264,2.044077,0.814341
Mornin' Delight,0.988747,0.383264,2.044077,0.814341
Adios Ghost,1.812704,0.702652,0.0,0.746479
Rare Scooop,0.906352,0.0,4.49697,0.0
Chemtrailmix - Rye Barrel,0.0,0.0,0.0,0.0



**Difference in Lifts Between "Barrel Aged Imperial German Chocolate Cupcake Stout" and the Other 9 Beers for the Top 4 Attributes**



| Beer Name | Taste | Carbonation | Chocolate | Lacing | Total Difference across all Attributes |
| --- | --- | --- | --- | --- |  --- |
| The Adjunct Trail - Bourbon Barrel-Aged | 0.517073 | 0 | 2.569697 | 0.639839 | 3.726609
| Twice the Daily Serving: Raspberry | 0.689431 | 0 | 2.569697 | 0.639839 | 3.898967
| Art | 0.387805 | 0 | 2.569697 | 0.639839 | 3.597341
| Barrel Aged Imperial German Chocolate Cupcake Stout | 0 | 0 | 0 | 0 | 0
| Truth - Vanilla Bean | 0.14102 | 0.38254 | 0.52562 | 0.174502 | 1.223682
| Mornin' Delight | 0.470067 | 0.38254 | 0.52562 | 0.174502 | 1.552729
| Adios Ghost | 1.292683 | 0.701323 | 2.569697 | 0.10664 | 4.670343
| Rare Scooop | 0.387805 | 0 | 1.927273 | 0.639839 | 2.954917
| Chemtrailmix - Rye Barrel | 0.517073 | 0 | 2.569697 | 0.639839 | 3.726609
| Continuance Blend #1 | 0.517073 | 1.683176 | 1.027879 | 0.255936 | 3.484064


We chose Barrel Aged Imperial German Chocolate Cupcake Stout to compare to the other 9 beers. We took the absolute values of the difference between each beer's lift values and the Imperial German Chocolate Cupcake Stout beer to get the difference in lift for each attribute. Then we took the sum of all of these differences for each beer and the beer with the lowest total difference was Truth - Vanilla Bean, with a difference of 1.22, meaning it is the most similar beer. Since a higher lift means we can have higher confidence in the importance of a rule, finding the lowest difference in lift attributes between the Barrel Aged Imperial German Chocolate Cupcake Stout and the rest of the beers would allow us to say that these two beers show/are associated with the same relative amounts of certain characteristics, making them the most similar. For example, a beer that has a "chocolate" lift of 1.84 would be most similar to a beer that also has a "chocolate" lift close to 1.84, say 1.96. This would result in a difference of 0.12, a very close difference, meaning they are comparable. This is further affirmed, since they both have names that would indicate they are beers that mimic dessert type flavors.